# SIGE Benchmark on DDIM

## Preparations
### Installation (This may take several minute)

In [ ]:
!pip install torch torchvision
!pip install git+https://github.com/lmxyy/sige.git
!pip install torchprofile wget tqdm ipyplot pyyaml easydict

### Clone the Repository

In [ ]:
!git clone https://github.com/lmxyy/sige.git

In [ ]:
import os
os.chdir("sige/diffusion")
print(os.getcwd())

### Create Configurations

In [ ]:
from argparse import Namespace

import torch
import yaml
from easydict import EasyDict

from utils import set_seed

with open("configs/church_ddim256-original.yml", "r") as f:
    config_vanilla = yaml.safe_load(f)
config_vanilla = EasyDict(config_vanilla)
with open("configs/church_ddim256-sige.yml", "r") as f:
    config_sige = yaml.safe_load(f)
config_sige = EasyDict(config_sige)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
config_vanilla.device = device
config_sige.device = device

# Build a dummy args
args = Namespace()
set_seed(0)  # for reproducibility, feel free to change the seed

### Create Models

Define Some helper functions for creating the models.

In [ ]:
from typing import Optional

from torch import nn

from download_helper import get_ckpt_path
from models.ddim_arch.unet import UNet
from models.ddim_arch.sige_fused_unet import SIGEFusedUNet
from models.ema import EMAHelper

def build_model(args, config):
    network: str = config.model.network
    if network == "ddim.unet":
        Model = UNet
    elif network == 'ddim.sige_fused_unet':
        Model = SIGEFusedUNet
    else:
        raise NotImplementedError("Unknown network [%s]!!!" % network)
    model = Model(args, config)
    model = model.to(device)

    if config.model.ema:
        ema_helper = EMAHelper(mu=config.model.ema_rate)
        ema_helper.register(model)
    else:
        ema_helper = None

    return model, ema_helper

def restore_checkpoint(model: nn.Module, ema_helper: Optional[EMAHelper], path: str):
    if isinstance(model, nn.DataParallel):
        model = model.module
    states = torch.load(path)
    model.load_state_dict(states["model"])
    if ema_helper is not None:
        if "ema" not in states:
            ema_helper.register(model)
        else:
            ema_helper.load_state_dict(states["ema"])
    return model, ema_helper

Build vanilla model. **It may take some time to download the model weights. Sometimes the downloading may get stuck. You could rerun the cell or download the weights manually.**

In [ ]:
# Vanilla DDIM Model
vanilla_model, vanilla_ema_helper = build_model(args, config_vanilla)
pretrained_path = get_ckpt_path(config_vanilla)
restore_checkpoint(vanilla_model, vanilla_ema_helper, pretrained_path)
vanilla_model.eval()
print('Vanilla model is built successfully!')

Build SIGE model. **It may take some time to download the model weights. Sometimes the downloading may get stuck. You could rerun the cell or download the weights manually.**

In [ ]:
# SIGE DDIM Model
sige_model, sige_ema_helper = build_model(args, config_sige)
pretrained_path = get_ckpt_path(config_sige)
restore_checkpoint(sige_model, sige_ema_helper, pretrained_path)
sige_model.eval()
print('SIGE model is built successfully!')

### Prepare Data
We have prepared two pairs of user edits in [`./assets`](./assets). Here, we use [`./assets/original.png`](./assets/original.png) as the original image and [`./assets/edited.png`](./assets/edited.png) as the edited image. You are free to use any other pairs of data either in our benchmark dataset (see [README.md](./README.md)) or created yourself.

In [ ]:
from PIL import Image
from torchvision import transforms

original_image_path = './assets/original.png'
edited_image_path = './assets/edited.png'

original_image = Image.open(original_image_path)
edited_image = Image.open(edited_image_path)

assert config_vanilla.data.image_size == config_sige.data.image_size
image_size = config_vanilla.data.image_size
resize = transforms.Resize(image_size)
original_image = resize(original_image)
edited_image = resize(edited_image)

Display the images.

In [ ]:
import ipyplot
import numpy as np

ipyplot.plot_images(
    (np.array(original_image), np.array(edited_image)), 
    ("Original", "Edited"),
    img_width=image_size
)

Convert the images to tensors.

In [ ]:
toTensor = transforms.ToTensor()
original_image = toTensor(original_image).unsqueeze(0).to(device)
edited_image = toTensor(edited_image).unsqueeze(0).to(device)

# Rescale the tensors to [-1, 1]
original_image = 2 * original_image - 1
edited_image = 2 * edited_image - 1

e = torch.randn_like(original_image)
x0s = torch.cat([original_image, edited_image], dim=0)
es = torch.cat([e, e], dim=0)

Compute the difference masks. `sige` has some helper functions for this.

In [ ]:
from sige.utils import compute_difference_mask, dilate_mask, downsample_mask

assert config_vanilla.sampling.eps == config_sige.sampling.eps
assert config_vanilla.sampling.mask_dilate_radius == config_sige.sampling.mask_dilate_radius

eps = config_vanilla.sampling.eps
mask_dilate_radius = config_vanilla.sampling.mask_dilate_radius

difference_mask = compute_difference_mask(original_image, edited_image, eps=eps)
difference_mask = dilate_mask(difference_mask, mask_dilate_radius)

# Downsample the mask to different resolutions
masks = downsample_mask(difference_mask, image_size // (2 ** (len(config_vanilla.model.ch_mult) - 1)))

Visualize the masks.

In [ ]:
def mask_to_image(mask: torch.Tensor):
    mask_numpy = mask.cpu().numpy()
    image = Image.fromarray(mask_numpy)
    image = image.resize((image_size, image_size))
    return image

mask_image = mask_to_image(difference_mask)
message = "Sparsity: %.2f%%" % (100 * difference_mask.sum() / difference_mask.numel())
ipyplot.plot_images((np.array(mask_image),), ("Difference Mask",), (message,), img_width=image_size)

print("Downsampled Masks")
arrays, labels, messages = [], [], []
for i, (k, v) in enumerate(masks.items()):
    image = mask_to_image(v)
    arrays.append(np.array(image))
    labels.append("Resolution: %dx%d" % (k[0], k[1]))
    messages.append("Sparsity: %.2f%%" % (100 * v.sum() / v.numel()))
ipyplot.plot_images(arrays, labels, messages, img_width=image_size)

## Test Models
### Quality Results

Define the DDIM sampler.

In [ ]:
from samplers.ddim_sampler import DDIMSampler as Sampler

# The same, the sampler should be the same for both the original model and SIGE model.
sampler = Sampler(args, config_vanilla)

Get some diffusion variables.

In [ ]:
def get_sampling_sequence(config, noise_level=None):
    if noise_level is None:
        noise_level = config.total_steps

    skip_type = config.sampling.skip_type
    timesteps = config.sampling.sample_steps

    if skip_type == "uniform":
        skip = noise_level // timesteps
        seq = range(0, noise_level, skip)
    elif skip_type == "quad":
        seq = np.linspace(0, np.sqrt(noise_level * 0.8), timesteps - 1) ** 2
        seq = [int(s) for s in list(seq)]
        seq.append(noise_level)
    else:
        raise NotImplementedError("Unknown skip type [%s]!!!" % skip_type)
    return seq

# The sampling sequence should be the same for both the original model and SIGE model.
seq = get_sampling_sequence(config_vanilla, noise_level=config_vanilla.sampling.noise_level)
ts = torch.full((x0s.size(0),), seq[-1], device=x0s.device, dtype=torch.long)  # The starting timestep
xts = sampler.get_xt_from_x0(x0s, ts, es)  # Preturb the image with the corresponding noise level
gt_x0, gt_e = x0s[:1], es[:1]  # Used for the mask trick in SDEdit to keep the unedited regions unchanged.

Start denoising.
* Denoising with the vanilla model.

In [ ]:
with torch.no_grad():
    vanilla_generate_x0s = sampler.denoising_steps(
        xts[1:], vanilla_model, seq, 
        gt_x0=gt_x0, gt_e=gt_e, 
        difference_mask=difference_mask
    )
    vanilla_result = (vanilla_generate_x0s[0] + 1) / 2

* Denoising with the SIGE model. Currently, `sige` only support caching the model for a single step. Therefore, we run a simple simulation to get the results: For every denoising step, we first denoise the noisy original image (`xts[:1]`) to cache the activations and run the actual sparse inference on `xts[1:]`.

In [ ]:
with torch.no_grad():
    # Need a pre-run to determine the data shape
    sige_model.set_mode("full")
    sige_model(original_image, torch.zeros(1, device=device, dtype=torch.float32))

    # Set the difference masks for the sparse inference. 
    # It will automatically reduce the masks to active indices.
    sige_model.set_masks(masks)
    
    sige_generate_x0s = sampler.denoising_steps(
        xts, sige_model, seq, 
        gt_x0=gt_x0, gt_e=gt_e, 
        difference_mask=difference_mask
    )
    sige_result = (sige_generate_x0s[1] + 1) / 2

Visualize these two images.

In [ ]:
vanilla_result = vanilla_result.clip(0, 1).permute(1, 2, 0).cpu().numpy()
vanilla_result = (vanilla_result * 255).astype(np.uint8)
vanilla_image = Image.fromarray(vanilla_result)

sige_result = sige_result.clip(0, 1).permute(1, 2, 0).cpu().numpy()
sige_result = (sige_result * 255).astype(np.uint8)
sige_image = Image.fromarray(sige_result)

In [ ]:
import ipyplot
import numpy as np

ipyplot.plot_images((np.array(vanilla_image), np.array(sige_image)), ("Vanilla", "SIGE"), img_width=image_size)

These two images should be very similar.

### Efficiency Results
First, let's profile the MACs of these two models.

In [ ]:
from torchprofile import profile_macs

# Create some dummy inputs
dummy_inputs = (x0s[:1], ts[:1])

with torch.no_grad():
    vanilla_macs = profile_macs(vanilla_model, dummy_inputs)

    # For the SIGE model, we need to first run it in the `full`` mode to cache the results.
    sige_model.set_mode("full")
    sige_model(*dummy_inputs)

    # Check to the `profile` mode to profile MACs. This mode is only for the MACs profiling.
    sige_model.set_mode("profile")
    sige_macs = profile_macs(sige_model, dummy_inputs)

In [ ]:
print('Vanilla MACs: %.3fG' % (vanilla_macs / 1e9))
print('SIGE MACs: %.3fG' % (sige_macs / 1e9))

SIGE model has a $\sim 3.3\times$ MACs reduction. Now let's measure the latency.

In [ ]:
import time

from tqdm import tqdm

# Change these numbers if they are too large for you.
warmup_times = 100
test_times = 100

def measure_latency(model: nn.Module):
    for i in tqdm(range(warmup_times)):
        model(*dummy_inputs)
        if device.type == "cuda":
            torch.cuda.synchronize()
    start_time = time.time()
    for i in tqdm(range(test_times)):
        model(*dummy_inputs)
        if device.type == "cuda":
            torch.cuda.synchronize()
    cost_time = time.time() - start_time
    return cost_time, cost_time / test_times

with torch.no_grad():
    vanilla_cost, vanilla_avg = measure_latency(vanilla_model)

    # As we have already cached some dummy results for the SIGE model, no need to rerun it in the `full` mode.
    # Check to the `sparse` mode to test the latency.
    sige_model.set_mode("sparse")
    sige_cost, sige_avg = measure_latency(sige_model)

print('Vanilla: Cost %.2fs Avg %.2fms' % (vanilla_cost, vanilla_avg * 1000))
print('SIGE: Cost %.2fs Avg %.2fms' % (sige_cost, sige_avg * 1000))